# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [106]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export BUCKET_STAGING_NAME=your_gcp_gs_bucket_staging_name` 
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [107]:
import tensorflow as tf
import tensorflow_datasets

from tensorflow.keras.utils import to_categorical

from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors
)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

from google.cloud import storage

import math
import numpy as np
import os
import glob
import time
from datetime import timedelta
import shutil
from datetime import datetime
import pickle
import re
import codecs
import json

from google.api_core.client_options import ClientOptions
from googleapiclient import discovery
from googleapiclient import errors

## Check configuration

In [108]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.3.0-rc2-23-gb36436b087 2.3.0


In [109]:
print(tf.keras.__version__)

2.4.0


In [110]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [111]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Import local packages

In [112]:
import preprocessing.preprocessing as pp
import utils.model_metrics as mm
import utils.model_utils as mu

In [113]:
import importlib
importlib.reload(pp);
importlib.reload(mm);
importlib.reload(mu);

## Check the local model [if needed]

In [114]:
savemodel_path = os.path.join(savemodel_dir, 'saved_model')
os.makedirs(savemodel_path, exist_ok=True)

In [115]:
model=tf.keras.models.load_model(os.path.join(savemodel_path, 'tf_bert_classification'))

In [116]:
# check the saved model
print('Model: {}'.format(model.name))
for i in os.listdir(os.path.join(savemodel_path,model.name)):
        print(' ',i)

Model: tf_bert_classification
  projector_config.pbtxt
  variables
  history
  history_per_step
  train
  saved_model.pb
  assets
  validation


In [117]:
model.summary()

Model: "tf_bert_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  167356416 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [118]:
source_bucket_name = savemodel_path+'/'+model.name

In [119]:
for root, dirs, files in os.walk(source_bucket_name):
    for name in files:
        if not 'history' in name:
            print(os.path.join('.../', name))

.../projector_config.pbtxt
.../saved_model.pb
.../variables.data-00000-of-00001
.../variables.index
.../keras_embedding.ckpt-1.index
.../checkpoint
.../keras_embedding.ckpt-0.index
.../events.out.tfevents.1587997306.Fabien-Tarrades-MacBook-Pro.local.20696.11162.v2
.../events.out.tfevents.1587997350.Fabien-Tarrades-MacBook-Pro.local.profile-empty
.../events.out.tfevents.1587999214.Fabien-Tarrades-MacBook-Pro.local.22120.11162.v2
.../keras_embedding.ckpt-1.data-00000-of-00001
.../keras_embedding.ckpt-0.data-00000-of-00001
.../local.trace
.../local.trace
.../events.out.tfevents.1587999352.Fabien-Tarrades-MacBook-Pro.local.22120.34079.v2
.../events.out.tfevents.1587997456.Fabien-Tarrades-MacBook-Pro.local.20696.34079.v2


The **variables** directory contains a standard training checkpoint (see the guide to training checkpoints).  
The **assets** directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.  
The **saved_model.pb** file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs.

In [251]:
#os.environ['MODEL_LOCAL']=savemodel_path+'/'+model.name
os.environ['MODEL_LOCAL']=savemodel_path+'/saved_model/'+model.name

SavedModels may contain multiple variants of the model (multiple v1.MetaGraphDefs, identified with the **--tag_set** flag to saved_model_cli), but this is rare. APIs which create multiple variants of a model include 

In [252]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve 

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [253]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 5)
      name: serving_default_input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## Copy the local model on GCP [if needed]

In [123]:
destination_bucket_name = 'saved_model/tf_bert_classification'
copy_model_gcp=False

In [124]:
# will take some time since the size of the model is 2 GB!
buket_name = os.environ['BUCKET_NAME']
if copy_model_gcp:
    mu.copy_local_directory_to_gcs(source_bucket_name, buket_name, destination_bucket_name)

## Use model stored on GCP

### Define the path of the model on GCP

In [243]:
# use model trainied with TPU
#os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/tf_bert_classification_tpu_2020_06_21_211701/saved_model/'+model.name

# use model trained one batch with TF 2.2.0
#os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/tf_bert_classification_cpu_2020_08_29_165313/saved_model/'+model.name
#os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/tf_bert_classification_cpu_2020_08_29_211749/saved_model/'+model.name
os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/tf_bert_classification_cpu_2020_08_30_105018/saved_model/'+model.name

### Check the structure of the model

In [244]:
%%bash
saved_model_cli show --dir $MODEL_GCP --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 5)
      name: serving_default_input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## Model serving setup

In [128]:
# Normal VM has a model size of 500 MB 
# For more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.

region_model = 'us-central1'
#region_model = 'europe-west4'
#region_model = 'europe-west1'
#region_model = 'europe-west6'

In [129]:
regional_endpoint=False
if region_model=='europe-west4':
    regional_endpoint=True
elif region_model=='us-central1':
    regional_endpoint=True
print(' Region: {} is a regional endpoint: {}'.format(region_model, regional_endpoint))

 Region: us-central1 is a regional endpoint: True


In [130]:
batch_pred=True
if batch_pred:
    regional_endpoint=False

In [131]:
project_name = os.environ['PROJECT_ID']
project_id = 'projects/{}'.format(project_name)
if not regional_endpoint:
    ai_platform_serving = discovery.build('ml', 'v1')
else:
    endpoint = 'https://'+region_model+'-ml.googleapis.com'
    client_options = ClientOptions(api_endpoint=endpoint)
    ai_platform_serving = discovery.build('ml', 'v1', client_options=client_options)
# to list all model
ai_platform_serving_global = discovery.build('ml', 'v1')

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is a

### Check models already deployed

In [132]:
request = ai_platform_serving_global.projects().models().list(parent=project_id)

In [133]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    if 'models' in response.keys():
        for i in response['models']:
            print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
            print('    descrition: {}'.format(i['description']))
            print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:
  Model 's name: tf_bert_classification_test_batch_us_central1:
    descrition: this is a model for test using a 2 GB Bert model
    regions: ['us-central1']
  Model 's name: tf_gcp_census_test_batch_us_central1:
    descrition: this is a model for test using census gcp code
    regions: ['us-central1']


### Create a new model

In [134]:
# defining the name of the model for online prediction
if batch_pred:
    name_model = model.name+'_test_batch_'+region_model.replace('-','_')
else:
    name_model = model.name+'_test_'+region_model.replace('-','_')
description_model = 'this is a model for test using a 2 GB Bert model'

# Create a dictionary with the fields from the request body.
request_dict = {'name': name_model,
                'regions': [region_model],
                'description': description_model,
                'labels': {'region': region_model} }

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().create(parent=project_id, 
                                                         body=request_dict)

In [135]:
request_dict 

{'name': 'tf_bert_classification_test_batch_us_central1',
 'regions': ['us-central1'],
 'description': 'this is a model for test using a 2 GB Bert model',
 'labels': {'region': 'us-central1'}}

In [136]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['description']))
    print('  regions: {}'.format(response['regions']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

There was an error creating the model. Check the details:
Field: model.name Error: A model with the same name already exists.


### Defined all parameters and upload our models

In [139]:
# defining the name of the model for online prediction
parentId = 'projects/{}/models/{}'.format(project_name, name_model)
# Normal VM has a model size of 500 MB for more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.
#region_model = 'us-central1'

model_binaries = os.environ['MODEL_GCP'] 
machine_type='n1-standard-2' # custom N1 machine
version = 'V1'
python_version = '3.7'
runtime_version = '2.2'

# Create a dictionary with the fields from the request body.
request_dict = {'machineType': machine_type,
                'runtimeVersion': runtime_version,
                'pythonVersion': python_version,
                'framework': 'TENSORFLOW',
                'description': description_model,
                'deploymentUri': model_binaries,
                'name': version
               }

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().versions().create(parent=parentId, 
                                                                    body=request_dict)

In [141]:
#request_dict

In [142]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['metadata']['version']['description']))
    print('  runtimeVersion: {}'.format(response['metadata']['version']['runtimeVersion']))
    print('  framework: {}'.format(response['metadata']['version']['framework']))
    print('  machineType: {}'.format(response['metadata']['version']['machineType']))
    print('  pythonVersion: {}'.format(response['metadata']['version']['pythonVersion']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

Name of the model: create_tf_bert_classification_test_batch_us_central1_V1-1598714227482:
  descrition: this is a model for test using a 2 GB Bert model
  runtimeVersion: 2.2
  framework: TENSORFLOW
  machineType: n1-standard-2
  pythonVersion: 3.7


### Check that the new modelal was deployed

In [143]:
request = ai_platform_serving.projects().models().list(parent=project_id)

In [144]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    for i in response['models']:
        print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
        print('    descrition: {}'.format(i['description']))
        print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:
  Model 's name: tf_bert_classification_test_batch_us_central1:
    descrition: this is a model for test using a 2 GB Bert model
    regions: ['us-central1']
  Model 's name: tf_gcp_census_test_batch_us_central1:
    descrition: this is a model for test using census gcp code
    regions: ['us-central1']


## Model serving inference

### Prepare data for online prediction for BERT

example of format:

```
{'instances': 
  [
    {'input_ids': [101, 143, 18267, 15470, 90395, ...], 
     'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, .....], 
     'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, .....]
     }, 
     {'input_ids': [101, 17664, 143, 30728, .........], 
      'attention_mask': [1, 1, 1, 1, 1, 1, 1, .......], 
      'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, ....]
      }
  ]
}

```

In [212]:
# load data
tfrecord_data_dir=data_dir+'/tfrecord/sst2/tf_bert_classification/tmp/test'
os.makedirs(tfrecord_data_dir, exist_ok=True)
valid_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/test_dataset.tfrecord')
valid_dataset = valid_files.map(pp.parse_tfrecord_glue_files)

In [240]:
print(valid_dataset)

<MapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>


In [213]:
# format data for online prediction
np_array = np.array(list(valid_dataset.as_numpy_iterator()))

In [214]:
data_prediction=[]

for el in np_array[0:10]:
    #instance={'input_ids': el[0]['input_ids'].tolist(), 'attention_mask': el[0]['attention_mask'].tolist(), 'token_type_ids': el[0]['token_type_ids'].tolist()}
    instance={'input_ids': el[0]['input_ids'].tolist()}
    print(len(el[0]['input_ids']))
    data_prediction.append(instance)

128
128
128
128
128
128
128
128
128
128


In [210]:
print(data_prediction[0:2])

[{'input_ids': [101, 10139, 10103, 18768, 45611, 21096, 17173, 16197, 10125, 11379, 10171, 10103, 14127, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'input_ids': [101, 11531, 143, 25737, 16229, 10139, 14226, 16037, 117, 10982, 12977, 10108, 10103, 13178, 18089, 11125, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}]


In [186]:
# load data
tfrecord_data_dir=data_dir+'/tfrecord/sst2/tf_bert_classification/valid/'
os.makedirs(tfrecord_data_dir, exist_ok=True)
valid_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/valid_dataset.tfrecord')
valid_dataset = valid_files.map(pp.parse_tfrecord_glue_files)

In [187]:
# format data for online prediction
np_array = np.array(list(valid_dataset.as_numpy_iterator()))

data_prediction=[]

for el in np_array[0:10]:
    #instance={'input_ids': el[0]['input_ids'].tolist(), 'attention_mask': el[0]['attention_mask'].tolist(), 'token_type_ids': el[0]['token_type_ids'].tolist()}
    instance={'input_ids': el[0]['input_ids'][0:5].tolist()}
    data_prediction.append(instance)

In [188]:
np.shape(data_prediction)

(10,)

In [189]:
print(data_prediction[0:2])

[{'input_ids': [101, 143, 18267, 15470, 90395]}, {'input_ids': [101, 17664, 143, 30728, 10563]}]


### Make online prediction

In [190]:
# model
version_name='V1'

# use the one above or define it below
#name_model='...'

# use the one above, be careful with regional endpoint
#ai_platform_serving='...'

parent = 'projects/{}/models/{}/versions/{}'.format(project_name, name_model, version_name)

# data for prediction
request_data = {"instances": data_prediction}
#request_data = {"instances": [{"dense_input":data_prediction}]}

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().predict(body=request_data, 
                                                 name=parent)

In [191]:
#parent

In [192]:
#request_data

In [193]:
# Make the call.
try:
    response = request.execute()
    print('predictions:')
    for i in response['predictions']:
        print('  {}'.format(i))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error making prediction. Check the details:')
    print(err._get_reason())

predictions:
  [0.0563784465, 0.258359343]
  [0.0974663571, 0.265995]
  [0.144064397, 0.146083474]
  [0.181236595, 0.167533115]
  [0.157815099, 0.218312696]
  [0.141223267, 0.279269218]
  [0.091098994, 0.33870545]
  [0.12836, 0.14179872]
  [0.203758404, 0.264894307]
  [0.213752791, 0.0863856673]


### Batch predictions

In [194]:
# load data
tfrecord_data_dir=data_dir+'/tfrecord/sst2/tf_bert_classification/valid'
os.makedirs(tfrecord_data_dir, exist_ok=True)
valid_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/valid_dataset.tfrecord')
valid_dataset = valid_files.map(pp.parse_tfrecord_glue_files)


In [195]:
# format data for online prediction
np_array = np.array(list(valid_dataset.as_numpy_iterator()))

In [198]:
# create a json file with the right format for online prediction
serving_data_dir=data_dir+'/serving/sst2'
os.makedirs(serving_data_dir, exist_ok=True)
json_file = serving_data_dir+'/input_predict_gcloud.json' 

with codecs.open(json_file, 'w', encoding='utf-8') as f:
    for el in np_array[0:10]:
        #instance={'input_ids': el[0]['input_ids'].tolist(), 'attention_mask': el[0]['attention_mask'].tolist(), 'token_type_ids': el[0]['token_type_ids'].tolist()}
        instance={'input_ids': el[0]['input_ids'][0:5].tolist()}
        json.dump(instance, f , sort_keys=True)
        f.write("\n")

In [199]:
destination_bucket_name = 'serving/sst2'
buket_name = os.environ['BUCKET_NAME']
mu.copy_local_directory_to_gcs(serving_data_dir, buket_name, destination_bucket_name)

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is a

serving/sst2/input_predict_gcloud.json
copy of the file(s) on gs:// done !


In [200]:
# bert
# Regional endpoint only support online prediction and AI explanations
name_model='tf_bert_classification_test_batch_us_central1'
#version_name='V1'
region_model='us-central1'
version_name=None
input_paths='gs://'+os.environ['BUCKET_NAME']+'/serving/sst2/input_predict_gcloud.json'
output_path='gs://'+os.environ['BUCKET_NAME']+'/batch_prediction_bert_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
data_format='TEXT'
max_worker_count=20
runtime_version=None

not_deployed=False
if not_deployed:
    runtime_version='2.1'
    #uri='gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/tf_bert_classification_tpu_2020_06_21_211701/saved_model/'+model.name
    uri='gs://'+os.environ['BUCKET_NAME']+'/census_20200706_194610/keras-job-dir/4/keras_export'
    signatureName='serving_default'

In [201]:
model_id = '{}/models/{}'.format(project_id, name_model)
if version_name is not None:
    version_id = '{}/versions/{}'.format(model_id, version_name)

# Make a jobName of the format "model_name_batch_predict_YYYYMMDD_HHMMSS"
timestamp = time.strftime('%Y%m%d_%H%M%S', time.gmtime())

job_id = '{}_{}'.format(name_model,timestamp)
# Start building the request dictionary with required information.
body = {'jobId': job_id,
        'predictionInput': {
            'dataFormat': data_format,
            'inputPaths': [input_paths],
            'outputPath': output_path,
            'region': region_model}
       }

# Use the version if present, the model (its default version) if not.
if not_deployed:
    body['predictionInput']['uri'] = uri
    body['predictionInput']['signatureName'] = signatureName
else:
    if version_name is not None:
        body['predictionInput']['versionName'] = version_id
    else:
        body['predictionInput']['modelName'] = model_id

# Only include a maximum number of workers or a runtime version if specified.
# Otherwise let the service use its defaults.
#if max_worker_count:
#    body['predictionInput']['maxWorkerCount'] = max_worker_count

if runtime_version:
    body['predictionInput']['runtimeVersion'] = runtime_version

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().jobs().create(parent=project_id,
                                                       body=body)

In [202]:
body

{'jobId': 'tf_bert_classification_test_batch_us_central1_20200829_165207',
 'predictionInput': {'dataFormat': 'TEXT',
  'inputPaths': ['gs://multilingual_text_classification/serving/sst2/input_predict_gcloud.json'],
  'outputPath': 'gs://multilingual_text_classification/batch_prediction_bert_2020_08_29_185201',
  'region': 'us-central1',
  'modelName': 'projects/axarevvicnonprod/models/tf_bert_classification_test_batch_us_central1'}}

In [203]:
# Make the call.
try:
    response = request.execute()
    print('job requested.')

    # The state returned will almost always be QUEUED.
    print('state : {}'.format(response['state']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error making prediction. Check the details:')
    print(err._get_reason())

There was an error making prediction. Check the details:
Field: runtime_version Error: The specified runtime version '2.2' is not supported or is deprecated.  Please specify a different runtime version.  See https://cloud.google.com/ml/docs/concepts/runtime-version-list for a list of supported versions


In [125]:
request.execute()

HttpError: <HttpError 500 when requesting https://ml.googleapis.com/v1/projects/axarevvicnonprod/jobs?alt=json returned "Internal error encountered.">

In [105]:
#!gcloud ai-platform jobs submit prediction 'tf_bert_classification_test_batch_us_central1_20200825_test_v2' \
#    --model 'tf_bert_classification_test_batch_us_central1' \
#    --input-paths 'gs://multilingual_text_classification/serving/sst2/input_predict_gcloud.json' \
#    --output-path 'gs://multilingual_text_classification/test_bert_v1' \
#    --region 'us-central1' \
#    --data-format 'TEXT'

In [103]:
#!gcloud ai-platform jobs describe tf_bert_classification_test_batch_us_central1_20200825_test_v2

In [104]:
#!gcloud ai-platform jobs stream-logs tf_bert_classification_test_batch_us_central1_20200825_test_v2